In [1]:
import requests
import pandas as pd
import json
import numpy as np
import os
from datetime import datetime, date
import csv
import time
import openpyxl
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
time_pre_data = [2024, 7, 8]
time_data = [2024, 7, 9]

In [3]:
target_csv_path = r"D:\Stock\pool\stock_pool_%s%s%s.xlsx" % (time_pre_data[0], str(time_pre_data[1]).zfill(2), str(time_pre_data[2]).zfill(2))
output_file_path = r"D:\Stock\analyze_result\%s%s%s.xlsx"%(time_data[0], str(time_data[1]).zfill(2), str(time_data[2]).zfill(2))

In [4]:
# Test basic API

# link = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=open_data&date=%s&stockNo=%s'%(20240301, 2354)
# df = pd.read_csv(link, encoding='utf_8_sig')
# df.to_csv(r'D:\Stock\daily_result\STOCK_DAY_2354.csv', encoding='utf_8_sig')

# link = 'http://www.tpex.org.tw/web/stock/aftertrading/daily_trading_info/st43_result.php?d=%s/%s/%s&stkno=%s'%(113, 3, 1, 6223)
# json_data = requests.get(link).json()
# columns = ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']
# df = pd.DataFrame(json_data['aaData'], columns=columns)
# df.to_csv(r'D:\Stock\daily_result\daily_trading_info_6223.csv', encoding='utf_8_sig')

In [5]:
def get_twse_stock_db_info():
    link = 'http://www.twse.com.tw/exchangeReport/BWIBBU_ALL?response=open_data'
    df = pd.read_csv(link, encoding='utf_8_sig')
    return df
    
def get_twse_stock_info(df, stock):
    target_data = df[df["股票代號"] == int(stock)]
    name = target_data.iloc[0]['股票名稱']
    priceEarningRatio = target_data.iloc[0]['本益比']
    yieldRatio = target_data.iloc[0]['殖利率(%)']
    priceBookRatio = target_data.iloc[0]['股價淨值比']
    name, priceEarningRatio, yieldRatio, priceBookRatio
    return name, priceEarningRatio, yieldRatio, priceBookRatio

def get_otc_stock_db_info():
    link = 'http://www.tpex.org.tw/openapi/v1/tpex_mainboard_peratio_analysis'
    json_data = requests.get(link).json()
    df = pd.DataFrame.from_records(json_data)
    return df
    
def get_otc_stock_info(df, stock):
    target_data = df[df['SecuritiesCompanyCode'] == stock]
    name = target_data.iloc[0]['CompanyName']
    priceEarningRatio = target_data.iloc[0]['PriceEarningRatio']
    dividendPerShare = target_data.iloc[0]['DividendPerShare']
    yieldRatio = target_data.iloc[0]['YieldRatio']
    priceBookRatio = target_data.iloc[0]['PriceBookRatio']
    name, priceEarningRatio, yieldRatio, priceBookRatio
    return name, priceEarningRatio, yieldRatio, priceBookRatio

In [6]:
def string_with_comma_to_int(x):
    return int(x.replace(",", ""))

def string_with_comma_to_float(x):
    try:
        return float(x.replace(",", ""))
    except:
        return 0
    
def string_to_float(x):
    try:
        return float(x)
    except:
        return 0
    
def vol_for_twse(x):
    try:
        return round(float(x.replace(",", ""))/1000)
    except:
        return 0

def moving_average(x, w):
    return np.convolve(x, np.ones(w), "valid") / w

def get_stock_volumn_price(yy, mm, dd, stock_tag):
    date_tag = date_tag = str(yy) + str(mm).zfill(2) + "01"
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=open_data&date=%s&stockNo=%s'%(date_tag, stock_tag)
    try:
        df = pd.read_csv(url, encoding='utf_8_sig')
    except:
        return None
    # ["日期","成交股數","成交金額","開盤價","最高價","最低價","收盤價","漲跌價差","成交筆數"]
    df_target = df[['日期', '成交股數', '收盤價', '開盤價', '漲跌價差']]
    df_target.iloc[:, 1] = df_target.iloc[:, 1].apply(vol_for_twse) # volumn, 成交張數 = 成交股數 / 1000
    df_target.iloc[:, 2] = df_target.iloc[:, 2].apply(string_to_float) # price
    df_target.iloc[:, 3] = df_target.iloc[:, 3].apply(string_to_float) # price
    df_target.iloc[:, 4] = df_target.iloc[:, 4].apply(string_to_float) # price
    df_target.rename(columns = {'成交股數':'成交張數'}, inplace = True)
    return df_target

def get_otc_stock_volumn_price(yy, mm, dd, stock_tag):
    yy = yy - 1911
    url = 'http://www.tpex.org.tw/web/stock/aftertrading/daily_trading_info/st43_result.php?d=%s/%s/%s&stkno=%s'%(yy, mm, dd, stock_tag)
    json_data = requests.get(url).json()
    # ['日期', '成交張數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']
    columns = ['日期', '成交張數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']
    df = pd.DataFrame(json_data['aaData'], columns=columns)
    df_target = df[['日期', '成交張數', '收盤價', '開盤價', '漲跌價差']]
    df_target.iloc[:, 1] = df_target.iloc[:, 1].apply(string_with_comma_to_float) # volumn
    df_target.iloc[:, 2] = df_target.iloc[:, 2].apply(string_with_comma_to_float) # price
    df_target.iloc[:, 3] = df_target.iloc[:, 3].apply(string_with_comma_to_float) # price
    df_target.iloc[:, 4] = df_target.iloc[:, 4].apply(string_with_comma_to_float) # price
    return df_target

def color_profit_loss(val):
    color = 'transparent'
    if val == -1:
        color = 'limegreen'
    elif val == 1:
        color = 'lightcoral'
    return f'background-color: {color}'

In [7]:
def get_stock_info(time_data, stock_tag, isOtc):
    year = time_data[0]
    month = time_data[1]
    day = time_data[2]
    year_pre = year
    month_pre = month
    month_pre = month_pre - 1
    df_pre = None
    if month_pre <= 0:
        month_pre = 12
        year_pre = year_pre - 1
    if isOtc:
        df_pre = get_otc_stock_volumn_price(year_pre, month_pre, "01", stock_tag)
        time.sleep(0.2)
        df = get_otc_stock_volumn_price(year, month, "01", stock_tag)
        time.sleep(0.2)
    else:
        df_pre = get_stock_volumn_price(year_pre, month_pre, "01", stock_tag)
        time.sleep(0.2)
        df = get_stock_volumn_price(year, month, "01", stock_tag)
        time.sleep(0.2)
    if df is None:
        return None
    # print(stock_tag)
    # if df_pre is None:
    #     print(stock_tag, isOtc, time_data)
    try:
        df = df_pre.append(df)
        df = df.reset_index()
    except:
        # print(stock_tag, isOtc, time_data)
        return None
    
    # get latest date
    data = None
    data_pre = None
    for i in range(1, len(df)):
        try:
            row_date = df.loc[i, "日期"]
            row_date = row_date.replace('*', "")
            row_yy = int(row_date.split('/')[0]) + 1911
            row_mm = int(row_date.split('/')[1])
            row_dd = int(row_date.split('/')[2])
            if row_yy <= year and row_mm <= month and row_dd <= day:
                data = df.loc[i]
                data_pre = df.loc[i-1]
        except:
            # print(row_date)
            continue
    return data, data_pre

### Get Stock Pool Data

In [8]:
# data_list = []
# with open(target_csv_path, newline='') as csvfile:
#     rows = csv.reader(csvfile)
#     row_idx = 0
#     for row in rows:
#         if row_idx == 0:
#             title = row
#         else:
#             data_list.append(row)
#         row_idx = row_idx + 1
df_pool = pd.read_excel(target_csv_path, converters={'推薦日期':str, '代號':str})
df_pool.drop(columns=df_pool.columns[0], axis=1, inplace=True)

### Analyze Profit Loss

In [9]:
analyze_result = []
for idx in df_pool.index:
    date_suggest = df_pool.loc[idx]['推薦日期']
    id = df_pool.loc[idx]['代號']
    name = df_pool.loc[idx]['名稱']
    price_suggest = float(df_pool.loc[idx]['推薦股價'])
    isOTC = int(df_pool.loc[idx]['是否上櫃'])
    stock_result = get_stock_info(time_data, id, isOtc = isOTC)
    if stock_result is None:
        print(time_data, id, isOTC)
        print("wrong")
        continue
    else:
        latest_data, latest_pre_data = stock_result[0], stock_result[1]
    latest_price = latest_data[-3]
    latest_price_diff = latest_data[-1]
    latest_pre_price = latest_pre_data[-3]
    diff_ratio =  round((latest_price / price_suggest - 1) * 100, 2)
    latest_diff_ratio =  round((latest_price_diff / latest_pre_price) * 100, 2)
    profit_loss = 0 # -1: stop loss, 0: continue, 1: stop profit
    if diff_ratio >= 5:
        profit_loss = 1
    if diff_ratio <= -3.75:
        profit_loss = -1
    analyze_result.append([date_suggest, id, name, isOTC, price_suggest, latest_price, profit_loss, latest_diff_ratio, diff_ratio])

### Write Excel File

In [ ]:
df = pd.DataFrame(analyze_result)
df.columns = ['推薦日期', '代號', '名稱', '是否上櫃', '推薦價格', '最新價格', '停損停利', '當日績效(%)', '累積績效(%)']
df = df.style.applymap(color_profit_loss, subset=['停損停利'])
if os.path.exists(output_file_path):
    print("File exist!!!")
else:
    df.to_excel(output_file_path, encoding='utf_8_sig')

In [ ]:
# output_file_path = r"D:\Stock\analyze_result\%s%s%s.csv"%(time_data[0], str(time_data[1]).zfill(2), str(time_data[2]).zfill(2))
# if os.path.exists(output_file_path):
#     print("File exist!!!")
# else:
#     with open(output_file_path, 'a', newline='') as csvfile:
#         writer = csv.writer(csvfile)
#         writer.writerows([time_data])
#         writer.writerows([['推薦日期', '代號', '名稱', '是否上櫃', '推薦價格', '最新價格', '停損停利', '績效(%)']])
#         writer.writerows(analyze_result)